In [1]:
import pandas as pd
import numpy as np

import os

import torch
import torch.nn as nn
from torchvision import transforms
import torchvision.models as models
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence

from tqdm import tqdm

import matplotlib.pyplot as plt
import time

import spacy
from PIL import Image

import warnings
warnings.filterwarnings('ignore')

In [2]:
data_path = '/kaggle/input/flickr8k/captions.txt'
df = pd.read_csv(data_path)
df.head()

,image,caption
0,1000268201_693b08cb0e.jpg,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg,A little girl in a pink dress going into a woo...


In [3]:
max_len = 0

for caption in df['caption'].to_list():
    if len(caption.split(' '))>max_len:
        max_len = len(caption.split(' '))

print(max_len)

38


In [4]:
spacy_eng = spacy.load("en_core_web_sm")

In [5]:
class Vocabulary:
    
    def __init__(self, freq_threshold):
        self.itos = {0:'<PAD>',1:'<SOS>',2:'<EOS>',3:'<UNK>'}
        self.stoi = {'<PAD>':0,'<SOS>':1,'<EOS>':2,'<UNK>':3}
        
        self.freq_threshold = freq_threshold
    
    def __len__(self):
        return len(self.stoi)

    def tokenize_caption(self,text):
        return [token.text.lower() for token in spacy_eng.tokenizer(text)]
    
    def build_vocabulary(self, sentence_list):
        frequencies = {}
        idx = 4
        
        for sentence in sentence_list:
            for word in self.tokenize_caption(sentence):
                if word not in frequencies:
                    frequencies[word] = 1
                else:
                    frequencies[word] += 1
                
                if frequencies[word] == self.freq_threshold and len(word)>1 and word.isalpha():
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1
                    
    
    def convert_to_vector(self, text):
        tokenized_text = self.tokenize_caption(text)
        return [self.stoi[token] if token in self.stoi else self.stoi['<UNK>'] for token in tokenized_text]

In [6]:
class FlickrDataset(Dataset):
    
    def __init__(self, root_dir, captions_file, transform = None, freq_threshold = 10):
        self.root_dir = root_dir
        self.transform = transform
        
        self.df = pd.read_csv(captions_file)
        self.df = self.df[:4000]
        
        self.imgs = self.df['image']
        self.captions = self.df['caption']
        
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(self.captions)
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        caption = self.captions[index]
        img_id = self.imgs[index]
        img = Image.open(os.path.join(self.root_dir,img_id)).convert('RGB')
        
        if self.transform is not None:
            img = self.transform(img)
        
        caption_vector = [self.vocab.stoi['<SOS>']]
        caption_vector += self.vocab.convert_to_vector(caption)
        caption_vector.append(self.vocab.stoi['<EOS>'])
        
        return img, torch.tensor(caption_vector)

class MyCollate:
    
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx
        
    def __call__(self, batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs,dim = 0)
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first = False, padding_value = self.pad_idx)
        return imgs,targets

In [7]:
class MyCollate:
    
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx
        
    def __call__(self, batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs,dim = 0)
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first = False, padding_value = self.pad_idx)
        return imgs,targets

In [8]:
torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
class EncoderCNN(nn.Module):
    
    def __init__(self, embed_size, train_CNN = False):
        super(EncoderCNN, self).__init__()
        
        self.train_CNN = train_CNN
        
        self.inception = models.inception_v3(pretrained = True, aux_logits = True)
        self.inception.fc = nn.Linear(self.inception.fc.in_features, embed_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, images):
        features = self.inception(images)[0]
        return self.dropout(self.relu(features))

class DecoderRNN(nn.Module):
    
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
        super(DecoderRNN, self).__init__()
        
        self.embed = nn.Embedding(vocab_size,embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, features, captions):
        embeddings = self.embed(captions)
        embeddings = torch.cat((features.unsqueeze(0), embeddings),dim = 0)
        hiddens,_ = self.lstm(embeddings)
        outputs = self.linear(hiddens)
        return outputs

class CNNtoRNN(nn.Module):
    
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
        super(CNNtoRNN, self).__init__()
        
        self.encoderCNN = EncoderCNN(embed_size)
        self.decoderRNN = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers)
    
    def forward(self, images ,captions):
        features = self.encoderCNN(images)
        outputs = self.decoderRNN(features,captions)
        return outputs
    
    def caption_image(self, image, vocabulary, max_length = 38):
        result_caption = []
        
        with torch.no_grad():
            
            x = self.encoderCNN(image).unsqueeze(0)
            states = None
            
            for _ in range(max_length):
                
                hidden, states = self.decoderRNN.lstm(x,states)
                output = self.decoderRNN.linear(hidden.squeeze(0))
                predicted = torch.argmax(output)
                result_caption.append(predicted.item())
                
                x = self.decoderRNN.embed(predicted).unsqueeze(0) # Prepare the output to input

                if vocabulary.itos[predicted.item()] == '<EOS>':
                    break
            
        return [vocabulary.itos[idx] for idx in result_caption]

In [10]:
transform = transforms.Compose(
        [
            transforms.Resize((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

dir_path = '/kaggle/input/flickr8k/Images'

dataset = FlickrDataset(root_dir = dir_path, captions_file = '/kaggle/input/flickr8k/captions.txt', transform=transform)

pad_idx = dataset.vocab.stoi['<PAD>']

loader = DataLoader(
    dataset=dataset,
    batch_size=64,
    shuffle=True,
    pin_memory=True,
    collate_fn=MyCollate(pad_idx=pad_idx)
)

In [17]:
load_model = False
save_model = False
train_CNN = False

embed_size = 256
hidden_size = 256
vocab_size = len(dataset.vocab)
num_layers = 1
learning_rate = 0.001
num_epochs = 100

In [18]:
model = CNNtoRNN(embed_size, hidden_size, vocab_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi['<PAD>'])
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [19]:
# Only fine tune the last layer of CNNs
for name, param in model.encoderCNN.inception.named_parameters():
    if 'fc.weight' in name or 'fc.bias' in name:
        param.requires_grad = True
    else:
        param.requires_grad = train_CNN

In [20]:
model.train()

for epoch in range(num_epochs):

    start_time = time.time()

    for idx, (imgs, caption) in tqdm(enumerate(loader),total = len(loader),leave=False):
        imgs = imgs.to(device)
        caption = caption.to(device)

        outputs = model(imgs, caption[:-1])
        loss = criterion(
            outputs.reshape(-1, outputs.shape[2]), caption.reshape(-1)
        )

        optimizer.zero_grad()
        loss.backward(loss)
        optimizer.step()
    
    time_taken = time.time() - start_time
    print(f'Epoch - {epoch+1}/{num_epochs}, Loss - {loss}, Time Taken - {time_taken/60}')

Epoch - 1/100, Loss - 3.011389970779419, Time Taken - 0.5257211208343506


Epoch - 2/100, Loss - 2.826110601425171, Time Taken - 0.5111935853958129


Epoch - 3/100, Loss - 2.7210309505462646, Time Taken - 0.5518576184908549


Epoch - 4/100, Loss - 2.6435046195983887, Time Taken - 0.5184993982315064


Epoch - 5/100, Loss - 2.363044023513794, Time Taken - 0.51342613697052


...

Epoch - 96/100, Loss - 0.6131618022918701, Time Taken - 0.5241391698519389


Epoch - 97/100, Loss - 0.6917980313301086, Time Taken - 0.5312514185905457


Epoch - 98/100, Loss - 0.6214895844459534, Time Taken - 0.5116109212239583


Epoch - 99/100, Loss - 0.6185522079467773, Time Taken - 0.506682030359904


Epoch - 100/100, Loss - 0.6077899932861328, Time Taken - 0.5273542324701945


In [23]:
torch.save(model,'/kaggle/working/entire_model.pth')